
# <span style="color:rgb(213,80,0)">Comparación del código de control de robots seriales</span>

Gutiérrez Espriella Moisés Ariel


Programas de control de robots.

## 1. scara\_tray\_line\_py.py
### **1.1 Resumen y Objetivo del Programa**

Este programa implementa un nodo de ROS 2 en Python para controlar un robot SCARA. El objetivo principal es generar y ejecutar una **trayectoria lineal** en el espacio de trabajo del robot. El programa define una línea recta entre dos puntos y publica las posiciones articulares necesarias para seguir esa trayectoria en un tópico de ROS 2, permitiendo que el robot (ya sea en simulación o físico) la ejecute.

#### **Conceptos de Robótica Aplicados:**

-  **Modelado Cinemático Inverso:** Aunque el código no calcula explícitamente la cinemática inversa, se basa en este principio para determinar las posiciones articulares (`joint_positions`) que corresponden a los puntos de la trayectoria lineal.
-  **Planeación de Trayectorias:** El programa implementa una interpolación lineal simple en el espacio articular para moverse entre las posturas que definen la línea.
-  **Control en Lazo Abierto:** Publica una secuencia de comandos de posición sin utilizar retroalimentación para verificar si el robot alcanzó cada punto intermedio.

### **1.2 Estructura y Componentes del Nodo ROS 2**

El programa está estructurado como una clase de Python que hereda de `Node` de la librería `rclpy`.


| **Componente** <br>  | **Implementación en el Código** <br>  | **Rol y Funcionamiento** <br>   |
| :-- | :-- | :-- |
| **Nodo ROS 2** <br>  | class Scara(Node): super().\_\_init\_\_('scara\_tray\_line\_py') <br>  | Inicializa el nodo con el nombre scara\_tray\_line\_py. <br>   |
| **Publisher** <br>  | self.publisher\_ = self.create\_publisher(JointState, 'joint\_states', 10) <br>  | Crea un publicador en el tópico joint\_states para enviar mensajes de tipo JointState. Este tópico es el estándar en ROS para comunicar las posiciones de las articulaciones. <br>   |
| **Timer** <br>  | self.timer = self.create\_timer(0.5, self.timer\_callback) <br>  | Establece un temporizador que ejecuta la función timer\_callback cada 0.5 segundos. Este mecanismo controla la frecuencia con la que se envían los comandos de movimiento. <br>   |
| **Definición de la Trayectoria** <br>  | self.trayectoria = \[...\] <br>  | Una lista de listas que contiene las posiciones angulares (en radianes) de las 4 articulaciones del robot SCARA para dos puntos clave: el inicio y el fin de la línea. <br>   |
| **Lógica de Publicación** <br>  | timer\_callback(self) <br>  | En cada llamada, esta función selecciona la siguiente postura de la trayectoria, la asigna a un mensaje JointState, actualiza el header.stamp con el tiempo actual y publica el mensaje. <br>   |

### **1.3 Análisis del Algoritmo de Trayectoria**

El algoritmo implementado es una secuencia simple que alterna entre dos configuraciones articulares para simular el movimiento a lo largo de una línea.


1.  **Inicialización:** Se define una trayectoria con dos puntos en el espacio articular:

-  `q1 = [0.0, 0.0, 0.0, 0.0]` (posición de inicio)
-  `q2 = [0.5, 0.5, 0.2, 0.0]` (posición final)

 **2. Ejecución Cíclica:**  La función `timer_callback` utiliza un contador (`self.i`) para alternar entre `q1` y `q2`.


 **3. Publicación:**  En cada paso, se publica la configuración articular seleccionada. El robot, al recibir estos mensajes, se moverá hacia las posiciones indicadas.


Este enfoque, aunque simple, demuestra el concepto de **control por puntos clave (waypoints)**, fundamental en la planeación de trayectorias.

## **`2. dofbot_sequence_py.py`**
### **2.1 Resumen y Objetivo del Programa**

Este programa es un nodo de ROS 2 diseñado para controlar el robot Dofbot, un brazo robótico educativo. El objetivo es ejecutar una **secuencia de movimientos predefinida**, moviendo el robot a través de una serie de posturas clave. Esta funcionalidad es la base para tareas más complejas como el "pick and place".


**Conceptos de Robótica Aplicados:**


-  **Cinemática Directa:** El programa opera directamente en el **espacio articular** (`joint space`), especificando los ángulos de cada una de las 5 articulaciones del robot.
-  **Control de Secuencias:** Implementa una lógica de máquina de estados simple para recorrer una lista de posturas deseadas, demostrando cómo se pueden programar tareas paso a paso.
-  **Control de Posición:** Envía comandos directos de posición a las articulaciones del robot.

### **2.2 Estructura y Componentes del Nodo ROS 2**

De forma similar al programa anterior, se basa en una clase que hereda de `Node`.


| **Componente** <br>  | **Implementación en el Código** <br>  | **Rol y Funcionamiento** <br>   |
| :-- | :-- | :-- |
| **Nodo ROS 2** <br>  | class Dofbot(Node): super().\_\_init\_\_('dofbot\_sequence\_py') <br>  | Inicializa el nodo con el nombre dofbot\_sequence\_py. <br>   |
| **Publisher** <br>  | self.publisher\_ = self.create\_publisher(JointState, 'joint\_states', 10) <br>  | Publica mensajes JointState en el tópico joint\_states para controlar las articulaciones del Dofbot. <br>   |
| **Timer** <br>  | self.timer = self.create\_timer(1.0, self.timer\_callback) <br>  | Ejecuta la función de control cada 1.0 segundos. Un intervalo más largo le da tiempo al robot para completar cada movimiento antes de recibir el siguiente comando. <br>   |
| **Definición de Posturas** <br>  | self.posturas = \[...\] <br>  | Una lista que contiene 5 posturas clave, donde cada postura es un vector de 5 ángulos (en radianes) correspondientes a las articulaciones del Dofbot. <br>   |
| **Lógica de Secuencia** <br>  | timer\_callback(self) <br>  | Esta función avanza a través de la lista self.posturas utilizando un contador. Publica una postura en cada llamada y reinicia el ciclo cuando llega al final de la secuencia. <br>   |

### **2.3 Análisis del Algoritmo de Secuencia**

El algoritmo es una demostración clara de cómo programar una tarea secuencial en robótica.


1.  **Definición de Secuencia:** Se establece una lista de 5 posturas articulares:

-  `p1 = [0.0, 0.0, 0.0, 0.0, 0.0]` (Posición "Home" o de reposo)
-  `p2 = [0.5, 0.0, 0.0, 0.0, 0.0]` (Movimiento de la base)
-  `p3 = [0.5, -0.5, 0.0, 0.0, 0.0]` (Movimiento del "hombro")
-  `p4 = [0.5, -0.5, -0.5, 0.0, 0.0]` (Movimiento del "codo")
-  `p5 = [0.5, -0.5, -0.5, 1.0, 0.0]` (Movimiento de la "muñeca")


 **2. Ejecución Cíclica:**  La función `timer_callback` se encarga de:

-  Seleccionar la postura actual de la lista.
-  Construir el mensaje `JointState` con los nombres y posiciones de las articulaciones.
-  Publicar el mensaje.
-  Incrementar el contador para pasar a la siguiente postura en la próxima ejecución.
-  Reiniciar el contador al llegar al final de la lista para que la secuencia se repita.


Este método es fundamental para la **programación fuera de línea** (`offline programming`), donde se definen los movimientos del robot antes de su ejecución.

